In [1]:
K, M = map(int, input("Enter K and M :").split())
def K_mers(read, K):
    bins = {}
    for idx in range(len(read)-K+1):         # total number of k-mers = L - K + 1 ; L = len(read)
        k_mer = read[idx:idx+K]              # slicing the read to get k-mer
        if k_mer not in bins:
            bins[k_mer]=[]
        bins[k_mer].append(idx)
    print(bins)
    return bins

def M_mers(read, M):
    m_mers = {'A':[], 'C':[], 'G':[], 'T':[]}

    for i in range(len(read)-M+1):
        m_mers[read[i]].append(read[i:i+M])
    
    print("read :", read)
    print(m_mers)
    return m_mers

def get_K_mer(read):
    print("Read :",read)
    
    K_mers(read, K)

def get_M_mer(read):
    M_mers(read, M)
    
read = [">SRR094926.sra.1", "TGGCGTCATTTTCCCCAATCA", ">SRR094926.sra.2", "TAGACTGTTGAAAGTGAACAAAACTGATGA"]

if __name__=="__main__":
    
    k_mer = list(map(get_K_mer, filter(lambda read: read[0]!=">", read)))
    print('M-mers -')
    m_mer = list(map(get_M_mer, filter(lambda read: read[0]!=">", read)))

Enter K and M :5 3
Read : TGGCGTCATTTTCCCCAATCA
{'TGGCG': [0], 'GGCGT': [1], 'GCGTC': [2], 'CGTCA': [3], 'GTCAT': [4], 'TCATT': [5], 'CATTT': [6], 'ATTTT': [7], 'TTTTC': [8], 'TTTCC': [9], 'TTCCC': [10], 'TCCCC': [11], 'CCCCA': [12], 'CCCAA': [13], 'CCAAT': [14], 'CAATC': [15], 'AATCA': [16]}
Read : TAGACTGTTGAAAGTGAACAAAACTGATGA
{'TAGAC': [0], 'AGACT': [1], 'GACTG': [2], 'ACTGT': [3], 'CTGTT': [4], 'TGTTG': [5], 'GTTGA': [6], 'TTGAA': [7], 'TGAAA': [8], 'GAAAG': [9], 'AAAGT': [10], 'AAGTG': [11], 'AGTGA': [12], 'GTGAA': [13], 'TGAAC': [14], 'GAACA': [15], 'AACAA': [16], 'ACAAA': [17], 'CAAAA': [18], 'AAAAC': [19], 'AAACT': [20], 'AACTG': [21], 'ACTGA': [22], 'CTGAT': [23], 'TGATG': [24], 'GATGA': [25]}
M-mers -
read : TGGCGTCATTTTCCCCAATCA
{'A': ['ATT', 'AAT', 'ATC'], 'C': ['CGT', 'CAT', 'CCC', 'CCC', 'CCA', 'CAA'], 'G': ['GGC', 'GCG', 'GTC'], 'T': ['TGG', 'TCA', 'TTT', 'TTT', 'TTC', 'TCC', 'TCA']}
read : TAGACTGTTGAAAGTGAACAAAACTGATGA
{'A': ['AGA', 'ACT', 'AAA', 'AAG', 'AGT', 'AAC', 